<a href="https://colab.research.google.com/github/Prajnahardikar/Final_Year_Project/blob/master/BCINet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
from numpy import array
import numpy as np
from tensorflow.keras import utils as np_utils
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, SeparableConv2D, DepthwiseConv2D, Reshape, BatchNormalization, Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

channels = 56
samples = 251
kernelLength = 64
F1 = 8
F2 = 16
D = 2
N = 2

df = pd.read_csv("/content/drive/My Drive/FYP/train_slice.csv")
df = df.iloc[:,1:57]
df = df.transpose()
df = array(df)
df = df.reshape(5440,df.shape[0],251)

df_test = pd.read_csv("/content/drive/My Drive/FYP/test_slice.csv")
df_test = df_test.iloc[:,1:57]
df_test = df_test.transpose()
df_test = array(df_test)
df_test = df_test.reshape(3400,df_test.shape[0],251)

X_train = df[:4320] #1 - 4320
X_validate = df[4320:] # 4320 - 5440
X_test = df_test

# X_train = df[:3808] #1 - 4320
# X_validate = df[3808:4896] # 4320 - 5440
# X_test = df[4896:]

df_y = pd.read_csv("/content/drive/My Drive/FYP/TrainLabels.csv")
df_y = df_y.iloc[:,1].to_numpy(dtype ='int32')

df_y_test = pd.read_csv("/content/drive/My Drive/FYP/true_labels.csv", header = None)
df_y_test = df_y_test.iloc[:,0].to_numpy(dtype ='int32')

y_train = df_y[:4320]
y_validate = df_y[4320:]
y_test = df_y_test

# y_train = df_y[:3808]
# y_validate = df_y[3808:4896]
# y_test = df_y[4896:]

model = Sequential()
model.add(Input(shape = (channels, samples)))
model.add(Reshape(target_shape = (1, channels, samples)))
model.add(Conv2D(F1, (1, kernelLength), padding = 'same',
                                    input_shape = (1, channels, samples), activation = 'linear'))
model.add(BatchNormalization(axis = 1))
model.add(DepthwiseConv2D((channels, 1), padding = 'valid', activation = 'linear',
                          depth_multiplier = D, data_format = 'channels_first',
        depthwise_constraint = max_norm(1.)))
model.add(BatchNormalization(axis = 1))
model.add(Activation('elu'))
model.add(AveragePooling2D((1, 4), data_format = 'channels_first'))
model.add(Dropout(0.25))
model.add(SeparableConv2D(F2,(1,16), padding = 'same', activation = 'linear', data_format = 'channels_first'))
model.add(BatchNormalization(axis = 1))
model.add(Activation('elu'))
model.add(AveragePooling2D((1, 8), padding = 'same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid', kernel_constraint = max_norm(0.25)))
    
model.compile(loss='binary_crossentropy', optimizer= 'adam' ,metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size =16  , epochs = 150, 
                         validation_data=(X_validate, y_validate))

model.summary()


Epoch 1/150
270/270 [==============================] - 2s 9ms/step - loss: 0.6363 - accuracy: 0.6907 - val_loss: 0.6505 - val_accuracy: 0.6536
Epoch 2/150
270/270 [==============================] - 2s 8ms/step - loss: 0.6014 - accuracy: 0.7208 - val_loss: 0.6456 - val_accuracy: 0.6536
Epoch 3/150
270/270 [==============================] - 2s 8ms/step - loss: 0.5905 - accuracy: 0.7227 - val_loss: 0.6596 - val_accuracy: 0.6527
Epoch 4/150
270/270 [==============================] - 2s 8ms/step - loss: 0.5771 - accuracy: 0.7236 - val_loss: 0.6636 - val_accuracy: 0.6527
Epoch 5/150
270/270 [==============================] - 2s 8ms/step - loss: 0.5675 - accuracy: 0.7208 - val_loss: 0.6629 - val_accuracy: 0.6554
Epoch 6/150
270/270 [==============================] - 2s 8ms/step - loss: 0.5556 - accuracy: 0.7241 - val_loss: 0.6810 - val_accuracy: 0.6518
Epoch 7/150
270/270 [==============================] - 2s 8ms/step - loss: 0.5456 - accuracy: 0.7225 - val_loss: 0.6907 - val_accuracy: 0.6482

In [3]:
probs       = model.predict(X_test)
print('Probs',probs)
probs = (probs>0.50)

print(probs)
y_pred = []
for i in probs:
  for j in i:
    if j==True:
      y_pred.append(1)
    else:
      y_pred.append(0)
print(y_pred)

from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test, y_pred)
print(cm)

acc         = np.mean(y_pred==y_test)
print("Classification accuracy: %f " % (acc))



Probs [[0.824805  ]
 [0.99999833]
 [0.9790475 ]
 ...
 [0.99974805]
 [0.9987625 ]
 [0.99999917]]
[[ True]
 [ True]
 [ True]
 ...
 [ True]
 [ True]
 [ True]]
[1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 

In [12]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred,pos_label=1)
f_measure = metrics.precision_recall_fscore_support(y_test, y_pred, average = 'weighted')
f_measure[0]
print("Precision : " , f_measure[0])
print("Recall : " , f_measure[1])
print("F-score : " , f_measure[2])
print("AUC : " , metrics.auc(fpr, tpr))


Precision :  0.584452857189791
Recall :  0.5514705882352942
F-score :  0.5649969016329955
AUC :  0.4961870077278936
